In [26]:
import pandas as pd
import pickle
from collections import Counter
import nltk
import re
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Lucas
[nltk_data]     Alponti\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
# Importando o objeto dataframe
with open(r'df-pos-processado-caracteres.pickle', 'rb') as picklefile:
    df = pickle.load(picklefile)

In [28]:
# Selecionado apenas as linhas assunto e texto tratado, e removendo as duplicadas
df = df[['Assunto', 'Texto tratado']]
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)

In [29]:
# Removendo, com base apenas na coluna 'Texto tratado', os valores repetidos
v = df['Texto tratado'].value_counts()
repetidos = df[df['Texto tratado'].isin(v.index[v.gt(1)])]

In [30]:
df.drop(list(repetidos.index), axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [31]:
# Tokenizando a coluna texto tratado
df['Tokenizadas'] = [str(t).split() for t in df['Texto tratado']]

In [32]:
# Obtendo as stopwords mais comuns para português
stopwords = nltk.corpus.stopwords.words('portuguese')

In [33]:
# Criando um dicionário, que terá melhor performance ao ser percorrido
dict_stopwords = Counter(stopwords)

In [34]:
# Removendo as stopwords
for i in range(len(df)):
    df['Tokenizadas'][i] = [t for t in df['Tokenizadas'][i] if dict_stopwords[t]==0 and len(t)>2]

In [35]:
# Criando uma lista com todas as palavras
lista_palavras = []
for row in df['Texto tratado']:
    for word in str(row).split():
        lista_palavras.append(word)

counts = Counter(lista_palavras)

In [36]:
# Selecionando as palavras pouco comuns (menos de 100 ocorrências)
lista_pouco_comuns = [elem for elem in counts.most_common() if elem[1]<=10]

In [37]:
# Verificando o percentual destas palavras em relação ao total
100*len(lista_pouco_comuns)/len(lista_palavras)

1.5207713697611012

In [38]:
# Removendo as palavras pouco frequentes
for i in range(len(df)):
    df['Tokenizadas'][i] = [t for t in df['Tokenizadas'][i] if counts[t]>10]

In [39]:
# Criando lista com todas as palavras para encontrar palavras para corrigir
lista_palavras_corrigir = []
for row in df['Tokenizadas']:
    for word in row:
        lista_palavras_corrigir.append(word)

counts2 = Counter(lista_palavras_corrigir)

In [40]:
# Percorrendo a lista de palavras, selecionando apenas as finalizadas em ã ou ç
# ou que possuem caracteres não convencionais (^~´`)
lista_palavras_corrigir = [t for t in lista_palavras_corrigir if re.search(r'[\^~´`]|ã$|ç$', t)]
lista_palavras_corrigir = list(set(lista_palavras_corrigir))

In [41]:
# Excluindo da lista palavras que não devem ser removidas
lista_palavras_corrigir.remove('manhã')
lista_palavras_corrigir.remove('mairiporã')
lista_palavras_corrigir.remove('irmã')
lista_palavras_corrigir.remove('guardiã')

In [42]:
# Removendo as palavras acima
dict_corrigir = Counter(lista_palavras_corrigir)

for i in range(len(df)):
    df['Tokenizadas'][i] = [t for t in df['Tokenizadas'][i] if dict_corrigir[t]==0 and len(t)>2]

In [43]:
# Escrevendo lista de termos jurídicos para serem removidos
lista_termos_juridicos = [
    "inciso",
    "incisos",
    "inc",
    "incs",
    "artigo",
    "artigos",
    "art",
    "arts",
    "parágrafo",
    "parágrafos",
    "par",
    "pars"
]

dict_termos_juridicos = Counter(lista_termos_juridicos)

for i in range(len(df)):
    df['Tokenizadas'][i] = [t for t in df['Tokenizadas'][i] if dict_termos_juridicos[t]==0]

In [44]:
# Removendo numerais romanos
for i in range(len(df)):
    df['Tokenizadas'][i] = [t for t in df['Tokenizadas'][i] if not re.findall(
                            "\\b(?=[MDCLXVIΙ])M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})([IΙ]X|[IΙ]V|V?[IΙ]{0,3})\\b\\.?",
                            flags=re.I,
                            string=t)
                            ]

In [45]:
# Removendo linhas em que todas as palavras foram removidas
df.drop(df[df['Tokenizadas'].apply(lambda x: len(str(x))==0)].index, axis=0, inplace=True)
df.reset_index(drop=True, inplace=True)

In [46]:
# Salvando o objeto dataframe
with open(r'df-pos-processado-palavras.pickle', 'wb') as picklefile:
    pickle.dump(df, picklefile, protocol=pickle.HIGHEST_PROTOCOL)

In [47]:
df

,Assunto,Texto tratado,Tokenizadas
0,Direito Tributário,recursos extraordinários ma apelaç ne ô í coma...,"[recursos, extraordinários, santo, andré, reco..."
1,Direito Tributário,imposto propriedade predial e territorial urba...,"[propriedade, predial, territorial, urbana, pr..."
2,Direito Tributário,acórdão tributo imposto predial e territorial ...,"[acórdão, tributo, predial, territorial, urban..."
3,Direito Tributário,acórdão mandado de segurança iptu progressivid...,"[acórdão, segurança, progressividade, paulo, e..."
4,Direito Tributário,da função social do imóvel para se tornar um t...,"[função, social, tornar, tributo, natureza, es..."
...,...,...,...
81407,Direito Civil,indenizatória contexto probatório que enseja o...,"[probatório, enseja, reconhecimento, responsab..."
81408,Direito Civil,acórdão processo ação indenizatóría afastada a...,"[acórdão, processo, ação, afastada, pretensão,..."
81409,Direito Civil,penhora direito de preferência anterioridade d...,"[penhora, preferência, anterioridade, penhora,..."
81410,Direito Civil,agravo df instrumento ação de indenização em f...,"[agravo, ação, fase, execução, assistência, au..."
